In [158]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

load_dotenv()
API_KEY = os.getenv('API_KEY')

client = OpenAI(api_key=API_KEY)

In [159]:
df = pd.read_csv('27-oct.csv')
df['summaries'] = None
df['theme'] = None
df['keywords'] = None
df['note'] = None

In [164]:
from pydantic import BaseModel
class AbstractSummary(BaseModel):
    keywords: list[str]
    summaries: list[str]
    note: str

prompt = """
Extract key findings from the abstract as bullet points, following these rules:

Formatting:
- Present only substantiated findings
- Skip bullet points entirely if the abstract is purely theoretical or does not present findings
- Limit to only the most important findings (typically 0-3 bullets)

Each bullet must:
- State a complete, specific conclusion (e.g., "AI chatbots foster creative collaboration by enabling anonymous idea sharing")
- Be independently understandable without context
- Use concise, precise, simple language
- Focus solely on verified results and findings
- Include metrics when available, but not required
- Include ONLY findings where AI is directly involved in the result
- Describes an AI-related element, explicitly name it as such (e.g. "Design guidelines" → "AI design guidelines", "Explanations" → "AI explanations", "The system's outputs..." → "The AI system's outputs...")
- Express each finding in Subject-Predicate-Object (SPO) format where possible

Do NOT include:
- Research methodology or process descriptions
- References to "this paper" or "this study"
- Vague comparisons without explanation
- Framework descriptions or conceptual models
- Hypotheses or future work

Sample bullets:
❌ "The system showed improved performance over baseline"
✅ "Virtual reality environments enable deeper emotional engagement in therapy sessions"
✅ "AI-assisted writing tools reduce cognitive load by managing document structure"

Output Format:
- Summaries: Clear, quantified bullet points [Follow rules from first section]
- Note [if no bullets extracted]: state the paper type, e.g.
  - "Workshop announcement"
  - "Conceptual framework"
  - "Design methodology"
  - "Technical specification"
  - "Systematic review"
  
In addition, extract 1-3 keywords from the abstract:
- Keywords [1-3]: Main topics and themes of the paper

include:
- Domain terms (e.g., "Healthcare", "Education")
- Target outcomes (e.g., "Team Efficiency", "Learning")
- Specific contexts (e.g., "Emergency Response", "K-12")

DO NOT include:
- Generic HCI/AI-related terms (e.g., "Human-AI Interaction", "Human-Computer Interaction", "Artificial Intellegence")
"""

def get_summary(abstract):
    chat_completion = client.beta.chat.completions.parse(
        messages=[
            {
                "role": "user",
                "content": prompt + " The following is the abstract: " + abstract,
            },
        ],
        model="gpt-4o",
        response_format=AbstractSummary,
        temperature=0.5
    )
    
    return chat_completion.choices[0].message.parsed

In [165]:
for i in range(len(df)):
    print(i, df.iloc[i]['title'])
    s = get_summary(df.iloc[i]['abstract'])
    df.at[i, 'summaries'] = s.summaries
    df.at[i, 'keywords'] = s.keywords
    df.at[i, 'note'] = s.note

0 Theory of Mind in Human-AI Interaction
1 Assessing Human-AI Interaction Early through Factorial Surveys: A Study on the Guidelines for Human-AI Interaction
2 Spiritual AI: Exploring the Possibilities of a Human-AI Interaction Beyond Productive Goals
3 Development and translation of human-AI interaction models into working prototypes for clinical decision-making
4 Exploration of Explainable AI for Trust Development on Human-AI Interaction
5 Re-examining User Burden in Human-AI Interaction: Focusing on a Domain-Specific Approach
6 A Systematic Review on Fostering Appropriate Trust in Human-AI Interaction: Trends, Opportunities and Challenges
7 Measuring User Experience Inclusivity in Human-AI Interaction via Five User Problem-Solving Styles
8 Human-AI interaction: intermittent, continuous, and proactive
9 Designing for Human-AI Interaction: Comparing Intermittent, Continuous, and Proactive Interactions for a Music Application
10 "Help Me Help the AI": Understanding How Explainability C

In [166]:
def print_summary(summary):
    # print(f"Theme: {summary.theme}")
    print()
    print(summary.title)
    print(f"Keywords: {summary.keywords}")
    print(f"Note: {summary.note}")
    for s in summary.summaries:
        print(f"- {s}")
        
for i in range(21,30):
    print_summary(df.iloc[i])


Effects of Communication Directionality and AI Agent Differences in Human-AI Interaction
Keywords: ['Human-AI Collaboration', 'Social Perceptions', 'Cooperative Games']
Note: 
- AI agents perceived as human increase rapport and likeability in cooperative games.
- AI communication direction affects perceived intelligence and game outcomes.

UX Research on Conversational Human-AI Interaction: A Literature Review of the ACM Digital Library
Keywords: ['Conversational Agents', 'Social Interactions', 'User Experience']
Note: 
- Polyadic conversational agents enhance human-human connection by facilitating communication and engagement.
- Designing polyadic conversational agents with social boundaries, such as privacy and disclosure, is crucial for ethical interactions.

Human-AI interaction and ethics of AI: how well are we following the guidelines
Keywords: ['AI Acceptance', 'Ethics Guidelines', 'Human-AI Interaction']
Note: 
- HAI interactions designed following AI interaction guidelines ma

In [168]:
df.to_csv('analyze.csv')